In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, TimeDistributed, multiply, Lambda, InputLayer
from tensorflow.keras.layers import LSTM, GRU, LeakyReLU, BatchNormalization, ReLU, concatenate, Masking, Permute, Reshape
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, RepeatVector,Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Nadam
from tensorflow.keras.initializers import *
from tensorflow.keras import Model
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
np.random.seed(73) 

def getmodel(n1, dp1, ac, opt, los, learn):
  mcdp = True
  
  if ac == 1:
    ac = 'relu'
  elif ac ==2:
    ac = 'selu'
  else: 
    ac = 'elu'

  if los ==1:
    loss = 'mse'
  elif los ==2:
    loss = 'mae'
  else: 
    loss = 'huber'
  
  if opt == 1:
    opt = Adam(lr=learn)
  elif opt == 2:
    opt = Nadam(lr=learn)
  else: 
    opt = RMSprop(lr = learn)
  input = Input(shape=(14,))
  dense = Dense(n1, activation=ac)(input)
  dense = Dropout(dp1)(dense, training=mcdp)
  dense = Dense(n1,activation=ac)(dense)
  dense = Dropout(dp1)(dense, training=mcdp)
  dense = Dense(n1,activation=ac)(dense)
  dense = Dropout(dp1)(dense, training=mcdp)
  output1 = Dense(1, activation='linear')(dense)

  model = Model(inputs= input, outputs=output1)
  #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

  #model.summary()
  print(dp1, ac, loss, opt, learn, n1)
  model.compile(optimizer=opt, loss=loss)
  return model 

In [ ]:
def remove_outlier(df_in, col_name):
  q1 = df_in[col_name].quantile(0.15)
  q3 = df_in[col_name].quantile(0.85)
  iqr = q3-q1 
  fence_low  = q1-1.5*iqr
  fence_high = q3+1.5*iqr
  df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
  return df_out

df = pd.read_csv('ropaxdata.csv', sep=',')
namarr = ['target','fuelDensity.csv','fuelTemp.csv','inclinometer-raw.csv', 'longitudinalWaterSpeed.csv','level1median.csv','level2median.csv', 'portPitch.csv', 'portRudder.csv','speedKnots.csv','starboardPitch.csv', 'starboardRudder.csv','trueHeading.csv', 'windAngle.csv', 'windSpeed.csv']
df.dropna(inplace=True)
df = df[namarr]
chosen_idx = np.random.choice(610406, replace=False, size=50)
df_trimmed = df.iloc[chosen_idx]
#print(df.isnull().sum())

for nam in namarr: 
  df = remove_outlier(df,nam)
print(df.shape)
df = df.sample(frac=0.5, replace=False, random_state=42, axis=0)
print(df.shape)
#sns.pairplot(data = df, vars=namarr)
#plt.show()
bounds = [{'name': 'l1',                'type': 'discrete',    'domain': (50,75,100)},
          {'name': 'l1_drop',          'type': 'continuous',  'domain': (0.2, 0.5)},
          {'name': 'learn',       'type': 'discrete',    'domain': (1e-2,1e-3,1e-4)}]        

In [ ]:
import time
import GPy, GPyOpt
import tqdm
import sklearn
from sklearn import model_selection

print(df.shape)
X = df.drop(['target'], axis=1).to_numpy()
Y = df['target'].to_numpy()
#X = sklearn.preprocessing.StandardScaler().fit_transform(X)
X = sklearn.preprocessing.MinMaxScaler().fit_transform(X)
#Y = np.divide(np.subtract(Y, Y.mean()), Y.std())
Y = np.divide(np.subtract(Y, np.min(Y)), np.max(Y)-np.min(Y))
Xtest, Xpool, ytest, ypool = sklearn.model_selection.train_test_split(X,Y,test_size=0.7, shuffle=True)
#Xptrain, Xptest, yptrain, yptest = model_selection.train_test_split(X,Y,test_size=0.3, shuffle=True)
addn=5000
order=np.random.permutation(range(len(Xpool)))
poolidx=np.arange(len(Xpool),dtype=np.int)
ninit = 100000 #initial samples
#initial training set
trainset=order[:ninit]
Xtrain=np.take(Xpool,trainset,axis=0)
ytrain=np.take(ypool,trainset,axis=0)
#remove data from pool
poolidx=np.setdiff1d(poolidx,trainset)

In [ ]:
import time
def uncerts(x):
  testacc_al=[]
  trainset=order[:ninit]
  Xtrain=np.take(Xpool,trainset,axis=0)
  ytrain=np.take(ypool,trainset,axis=0)
  poolidx=np.arange(len(Xpool),dtype=np.int)
  poolidx=np.setdiff1d(poolidx,trainset)
  model = getmodel(int(x[0]),  float(x[1]),  int(x[2]), int(x[3]),  int(x[4]),  float(x[5]))
  for i in range(20):
    #fit model
    history = model.fit(Xtrain,ytrain,batch_size=int(x[6]), epochs=int(x[7]), verbose=0) 
    r2_arr = []
    pred = []
    #predict on test set
    for j in tqdm.tqdm(range(50)):
      yhat = model.predict(Xtest, batch_size=int(x[6]))
      r2_arr.append(sklearn.metrics.r2_score(ytest,yhat))
      yhat2 = model.predict(Xpool[poolidx], batch_size=128)
      pred.append(yhat2)
    eval = np.mean(r2_arr)
    print(eval)
    print(len(Xtrain))
    pred = np.array(np.squeeze(pred)).T
    std = []
    for rw in pred:
      std.append(np.std(rw))
    std = np.array(std)
    testacc_al.append((len(Xtrain),eval))
    #select least confident max likely label - then sort in negative order - note the minus
    ypool_p_sort_idx = np.argsort(std)
    #add to training set
    Xtrain=np.concatenate((Xtrain,Xpool[poolidx[ypool_p_sort_idx[-addn:]]]))
    ytrain=np.concatenate((ytrain,ypool[poolidx[ypool_p_sort_idx[-addn:]]]))
    #remove from pool
    poolidx=np.setdiff1d(poolidx,ypool_p_sort_idx[-addn:])
  return model, testacc_al


model, test = uncerts([5.00e+01, 1.00e-01, 1.00e+00, 3.00e+00, 1.00e+00, 1.00e-03, 1.28e+02, 3.80e+01])  
model.save('uncer.h5')
print(test)
"""
start = time.time()
opt = GPyOpt.methods.BayesianOptimization(f=uncerts, domain=bounds, maximize=True)

# optimize  model
opt.run_optimization(max_iter=5)
print(opt.x_opt, opt.fx_opt)
end = time.time()
print(end - start)"""

In [ ]:
print(test)
model.save('uncertain.h5')